In [1]:
###
### PYTHON PACKAGES
###
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy
from spacy import displacy

###
### USER-DEVELOPED IMPORTS
###
# from project import *
from capture_group import *

game change | best miniseries | winner
the hour | best miniseries | nominee
argo | best drama | winner


### New Idea

1) loop through cleaned dataframe to find tweets with "win," "present," "nominate," "best" (maybe), "award" (maybe)
2) with a match, get any names

In [2]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']
win_data = pd.read_csv('wins.csv')['text']
spacy_data = pd.read_csv('spacy_info.csv')

In [3]:
spacy_size = 1500

In [4]:
# win_data.head(50).apply(lambda x: find_award_winner(x))x
find_award_winner("I hope Yash or Sorie wins best student ")

'Yash | best student | nominee'

In [5]:
def nominee_search(text):
    doc = nlp(text)
    
    entities = extract_entities(doc)
    
    gen_patterns = r'(is nominated for|is a nominee for)'
    if re.search(gen_patterns, text, re.IGNORECASE):
        for ent in entities:
            if ent[1] == "PERSON":
                alleged_winner = ent[0]
        
        award = extract_award_name_after_best(doc)
        
        if award != None:
            return alleged_winner, award
        
        word_list = ["award", "prize", "honor", "medal", "trophy"]
        pattern = r"(\w+)\s+" + gen_patterns + r"(.*?\b(?:" + "|".join(re.escape(word) for word in word_list) + r")\b)"
        match = re.search(pattern, text, re.IGNORECASE)
        
        return alleged_winner, match.group(3)

In [6]:
nominee_search("Yash is a nominee for best student")

NameError: name 'extract_entities' is not defined

### Notes from office hours

There may be accounts (i.e. newspapers) that HAVE to post who the nominees are, look through data for that
Can use ordering of events (i.e. award A goes Speaker -> Nominees -> Winner)
Start by looking at data that has information you know you need
Try inputting JSON into ChatGPT and ask it to return tweets where nominees for awards are mentioned
Somebody mentioned frequently near mention of an award winner

For Spacy, find nsubj+, check IMDB to see if it's actually a movie or actor/actress (if time)

Based on award name, can tell if you need a movie or actor / actress / song

See what Spacy outputs for "movie wins x award"


In [ ]:
from itertools import islice

account_rts = {}

for text in cleaned_data:
    tweet = text.split(' ')
    if tweet[0] == 'rt':
        if tweet[1] in account_rts:
            account_rts[tweet[1]] += 1
        else:
            account_rts.update({tweet[1]:1})

account_dicts = dict(sorted(account_rts.items(), key=lambda item: item[1], reverse=True))

top_accounts = list(islice(account_dicts, 10))
top_accounts

print(top_accounts)

['@goldenglobes', '@eonline', '@perezhilton', '@theellenshow', '@emwatson', '@vanityfair', '@nbcsnl', '@cnnshowbiz', '@ciudadbizarra', '@ew']


In [ ]:
retweet_pattern = r're\s' + '|'.join(top_accounts)
match = re.search(retweet_pattern, text, re.IGNORECASE)

In [ ]:
def account_rt(text):
    doc = nlp(text)
    print(text, " --- 1")
    
    pattern = r're \s' + '|'.join(top_accounts)
    
    if re.search(pattern, text, re.IGNORECASE):     
        print(text, " --- 2")   
        return find_award_winner(text)

In [ ]:
text = "rt @goldenglobes best screenplay quentin tarantino django unchained"

In [ ]:
account_rt(text)

rt @goldenglobes best screenplay quentin tarantino django unchained  --- 1


In [ ]:
df = pd.read_json('gg2013.json')['text']

In [ ]:
df[102541]

'RT @goldenglobes: Best Screenplay - Quentin Tarantino - Django Unchained - #GoldenGlobes'

In [12]:
nominee_data = cleaned_data[cleaned_data.apply(lambda x: re.search(r'(\bnominated\b|\bnominee\b|\bnomination\b|nominated\b)', x) != None)]

nominee_data.to_csv("nominees.csv")

In [13]:
nominee_data

46        RT @Loki_Page: #GoldenGlobes Times! We wish go...
132       I've seen a grand total of one nominated film....
290       RT @Zap2itRick: Can Kerry Washington be nomina...
391       If Leo could wear a tux everyday ... He still ...
629       I wish my dad was an award-winning/nominated a...
                                ...                        
152540    RT @JessicaCapshaw: Thank u all for your play ...
152550    RT @VanityFair: Best nominee cutaway of the ni...
152644    RT @JessicaCapshaw: Thank u all for your play ...
152707    RT @oskrNYC: "7 Cajas" should have been nomina...
152787    RT @MaPeel: #Skyfall. Yes. Should have been no...
Name: text, Length: 1023, dtype: object

In [14]:
win_data = pd.read_csv('wins.csv')['text']
win_output = win_data.apply(find_award_winner)

In [15]:
nominee_data = pd.read_csv('nominees.csv')['text']
nom_output = nominee_data.apply(find_nominee)

In [16]:
pd.concat([win_output, nom_output]).to_csv('winners_nominees.csv', index=False)

In [ ]:
winner = ['damien', 'anne', 'amy']

re.search(f'{winner}+but I love+')

In [108]:
answers = pd.read_csv('winners_and_awards.csv')['text']
answers

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
4072                   Hugh Jackman | best actor | winner
4073                   Hugh Jackman | Best Actor | winner
4074                                                  NaN
4075    a video Jennifer Lawrence | Best Actress | winner
4076        Les Miserables | Best Motion Picture | winner
Name: text, Length: 4077, dtype: object

In [109]:
# award class to store information about each award
class award:
    def __init__(self, name):
        self.name = name
        self.winner = {}
        self.nominees = {}
        self.presenters = {}
        self.votes = 1
    
    # add a new person associted with the award
    def new_person(self, name, role):
        if role == 'winner':
            self.winner.update({name:1})
        elif role == 'nominee':
            self.nominees.update({name:1})
        elif role == 'presenter':
            self.presenters.update({name:1})
    
    # a repeat-appearence of a person in a role gives them an additional vote for that role (more votes = more likely)
    def person_vote(self, name, role):
        if role == 'winner':
            self.winner[name] += 1
        elif role == 'nominee':
            self.nominees[name] += 1
        elif role == 'presenter':
            self.presenters[name] += 1
    
    # a repeat award gets more votes (more votes = more likely)
    def award_vote(self):
        self.votes += 1
    
    # check if a person is already present in a particular award
    def contains(self, name, role):
        if role == 'winner':
            return name in self.winner
        elif role == 'nominee':
            return name in self.nominees
        elif role == 'presenter':
            return name in self.presenters
    
    def output(self):
        return {self.name:
                { 
                    "nominees": [(n, self.nominees[n]) for n in self.nominees],
                    "presenters":[(p, self.presenters[p]) for p in self.presenters], 
                    "winner":[(w, self.winner[w]) for w in self.winner],
                    "votes":self.votes
                }
        }

In [114]:
# list of awards and their respective winners, nominees, and presenters, along with the number of times they appear in our data
awards = {}

# Split up each row of our dataframe and apply appropriate function to update our answers
def extract_answers(text):
    nominee, curr_award, role = text.split(' | ')
    
    if curr_award not in awards:
        awards.update({curr_award:award(curr_award)})
    else:
        awards[curr_award].award_vote()
        
    if not awards[curr_award].contains(nominee, role):
        awards[curr_award].new_person(nominee, role)
    else:
        awards[curr_award].person_vote(nominee, role)

In [115]:
answers = answers.dropna()

In [116]:
answers.apply(extract_answers)

18      None
25      None
26      None
30      None
36      None
        ... 
4071    None
4072    None
4073    None
4075    None
4076    None
Name: text, Length: 2729, dtype: object

In [117]:
for a in awards:
    print(awards[a].output())

{'best personality': {'nominees': [], 'presenters': [], 'winner': [('Dustin Hoffman', 1)], 'votes': 1}}
{'best': {'nominees': [], 'presenters': [], 'winner': [('he', 1), ('Tina Fey', 1), ('Christoph Waltz', 52), ('# GoldenGlobes', 2), ('christoph Waltz', 1), ('Christopher Waltz', 1), ('Cristoph Waltz', 1), ('Maggie Smith', 5), ('Julianne', 1), ('who', 1), ('Adele', 1), ('LINCOLN', 1), ('GoldenGlobes Ed Harris', 1), ('Ed Harris', 3), ('Anne Hathaway', 75), ('GoldenGlobes Anne Hathaway', 1), ('student Anne Hathaway', 1), ('# AnneHathaway', 1), ('Hathaway', 1), ('Brave', 18), ('# Brave', 16), ('None', 1), ('GoldenGlobes', 2), ('# Goldenglobes', 2), ('DDL', 1)], 'votes': 191}}
{'the beard award': {'nominees': [], 'presenters': [], 'winner': [('Bill Murray', 1)], 'votes': 1}}
{'an award': {'nominees': [('GoldenGlobes', 1), ('someone', 1), ('hope', 1)], 'presenters': [], 'winner': [('Christoph Waltz', 2), ('a Dowager Countess', 1), ('Everytime Life', 1), ('she', 4), ('Who', 1), ('Anne Hathaw